In [2]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import numpy as np
import sys
sys.path.append("..")
from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.visualization import show_network, show_demand
from dyntapy.assignments import StaticAssignment

In [3]:
import geopandas as gpd
from sklearn.cluster import DBSCAN
from shapely.geometry import Point


shapefile = gpd.read_file('STA_prep/shapefile_data/BRUSSEL_40_10/BRUSSEL_40_10.shp')
points = shapefile.centroid
# Convert points to a numpy array of coordinates
points_array = np.array(points.apply(lambda p: [p.x, p.y]).tolist())

dbscan = DBSCAN(eps=0.1, min_samples=5) # adjust these parameters as needed
clusters = dbscan.fit_predict(points_array)
shapefile['cluster'] = clusters
aggregated = shapefile.dissolve(by='cluster')

mean_x = np.mean(points_array[:,0])
mean_y = np.mean(points_array[:,1])
center = Point(mean_x, mean_y) 
distances = points.distance(center)
eps_values = 0.1 * distances / distances.max() # adjust the 0.1 factor as needed
clusters = DBSCAN(eps=0.1, min_samples=2).fit_predict(points_array)

aggregated.to_file("STA_prep/shapefile_data/BRUSSEL_40_10/BRUSSEL_40_10_aggr.shp", driver='ESRI Shapefile')


In [19]:
type(eps_values.astype(float))

pandas.core.series.Series

In [6]:
aggregated.centroid

cluster
-1    POINT (149555.586 172020.303)
dtype: geometry

In [5]:
print((mean_x,mean_y))

(149136.28344398143, 177490.1557429837)
